In [2]:
import pandas as pd
import numpy as np

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn import preprocessing, cross_validation
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

### Dataset contains around 2345796 data values

In [3]:
df = pd.read_csv('test.csv')
df.drop(['test_id'],1,inplace=True)
df = df.dropna()

In [4]:
stop_words = list(set(stopwords.words("english")))

In [5]:
question1 = [word_tokenize(i.lower().decode('utf8')) for i in df['question1'][0:10000]]
question2 = [word_tokenize(i.lower().decode('utf8')) for i in df['question2'][0:10000]]

In [6]:
q1 = [w for w in question1 if w not in stop_words]
q2 = [w for w in question2 if w not in stop_words]

In [7]:
q1 = [' '.join(w) for w in q1]
q2 = [' '.join(w) for w in q2]

In [8]:
df['question1'][0:10000] = q1
df['question2'][0:10000] = q2

In [9]:
quora = zip(q1,q2)

### Manual validation (need to take care)

In [10]:
try:
    label = []
    for i in quora:
        values = [j.split(" ") for j in i]
        compare = values
        same = [i for i in compare[0] if i in compare[1]]
        if len(same) > 5:
            label.append(1)
        else:
            label.append(0)
except AttributeError, UnicodeDecodeError:
    print('Error occured')

### Convert to numeric data

In [11]:
def handle_non_numerical_data(df):
    columns = df.columns.values

    for column in columns:
        text_digit_vals = {}

        def convert_to_int(val):
            return text_digit_vals[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = list(set(column_contents))
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x += 1

            df[column] = list(map(convert_to_int,df[column]))

    return df

### Split data

In [12]:
df = df[0:10000]
df = handle_non_numerical_data(df)

ft = np.array(df)
lt = np.array(label)

features_train,features_test,labels_train,labels_test = cross_validation.train_test_split(ft,lt,test_size=0.2)

### Gaussian Naive Bayes

In [13]:
clf = GaussianNB()
clf.fit(features_train,labels_train)
pred = clf.predict(features_test)
print(pred)
accuracy = accuracy_score(labels_test,pred)
print(accuracy)

[0 0 0 ..., 0 0 0]
0.6275


### AdaBoostClassifier

In [14]:
clf = AdaBoostClassifier()
clf.fit(features_train,labels_train)
pred = clf.predict(features_test)
print(pred)
accuracy = accuracy_score(labels_test,pred)
print(accuracy)

[0 0 0 ..., 0 0 0]
0.6255


### RandomForestClassifier

In [15]:
clf = RandomForestClassifier()
clf.fit(features_train,labels_train)
pred = clf.predict(features_test)
print(pred)
accuracy = accuracy_score(labels_test,pred)
print(accuracy)

[1 0 1 ..., 0 0 0]
0.5625


### KNeighborsClassifier

In [16]:
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(features_train,labels_train)
pred = clf.predict(features_test)
print(pred)
accuracy = accuracy_score(labels_test,pred)
print(accuracy)

[1 0 1 ..., 0 0 0]
0.5715
